In [1]:
import mxnet as mx

/mnt/D/Ubuntu/package/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


首先进行index啊, 对数据进行转换, 对各个维度进行统计, 最好用他们自带的包, 不然效率不高
实际生产用spark进行
这里用自己的测试数据手工代替

In [2]:
test_data = "/home/elliottqian/Documents/PycharmProjects/deeplearning_notebook/mxnet_study/movielens/test_data"

In [127]:
class YearOld(object):
    item_to_index = {"小孩子": 0, "大老人": 1}
    index_to_item = {0: "小孩子", 1: "大老人", 2: "others"}
    
    @staticmethod
    def get_index(x):
        if x in YearOld.item_to_index.keys():
            return YearOld.item_to_index[x]
        else:
            return len(YearOld.item_to_index)

In [128]:
class GameName(object):
    item_to_index = {"吃鸡游戏": 0, "追梦电影": 1, "梦幻西游": 2}
    index_to_item = {0: "吃鸡游戏", 1: "追梦电影", 2: "梦幻西游", 3: "others"}
    
    @staticmethod
    def get_index(x):
        if x in GameName.item_to_index.keys():
            return GameName.item_to_index[x]
        else:
            return len(GameName.item_to_index)

In [129]:
class ItemType(object):
    item_to_index = {"游戏": 0, "电影": 1}
    index_to_item = {0: "游戏", 1: "电影", 2: "others"}
    
    @staticmethod
    def get_index(x):
        if x in ItemType.item_to_index.keys():
            return ItemType.item_to_index[x]
        else:
            return len(ItemType.item_to_index)

In [130]:
class Sex(object):
    item_to_index = {"男": 0, "女": 1}
    index_to_item = {0: "男", 1: "女", 2: "0"}
    
    @staticmethod
    def get_index(x):
        try:
            return Sex.item_to_index[x]
        except _:
            return len(Sex.item_to_index)

In [131]:
import codecs
def covert(in_put):
    result = []
    labels = []
    with codecs.open(in_put, encoding="utf-8") as f:
        for line in f:
            line = line.strip().split(" ")
            print(line)
            label = int(line[0])
            sex = Sex.get_index(line[1])
            year_old = YearOld.get_index(line[2])
            name = GameName.get_index(line[3])
            type = ItemType.get_index(line[4])
            labels.append(label)
            result.append([sex, year_old, name, type])
    return result, labels

In [132]:
out_path = "/home/elliottqian/Documents/PycharmProjects/deeplearning_notebook/mxnet_study/movielens/out"

In [133]:
result, labels = covert(test_data)
result

['1', '男', '小孩子', '吃鸡游戏', '游戏']
['0', '女', '大老人', '追梦电影', '电影']
['1', '男', '小孩子', '梦幻西游', '游戏']
['1', '女', '小孩子', '吃鸡游戏', '游戏']


[[0, 0, 0, 0], [1, 1, 1, 1], [0, 0, 2, 0], [1, 0, 0, 0]]

In [96]:
labels

[1, 0, 1, 1]

In [10]:
input_dims = [len(Sex.index_to_item), len(YearOld.index_to_item), len(GameName.index_to_item), len(ItemType.index_to_item)]
input_dims

[3, 3, 4, 3]

In [11]:
result

[[0, 2, 2, 2], [1, 2, 2, 2], [0, 2, 2, 2], [1, 2, 2, 2]]

## make the train data and label to mx.nd.array type

In [135]:
train_data = mx.nd.array(result)
train_data


[[ 0.  0.  0.  0.]
 [ 1.  1.  1.  1.]
 [ 0.  0.  2.  0.]
 [ 1.  0.  0.  0.]]
<NDArray 4x4 @cpu(0)>

In [13]:
label_data = mx.nd.array(labels)

In [14]:
label_data


[ 1.  0.  1.  1.]
<NDArray 4 @cpu(0)>

In [15]:
from mxnet import gluon
import mxnet.gluon.nn as gnn


In [155]:
class MyEmbedding(gnn.Block):
    
    def __init__(self, dims, out_put_dim, **kwargs):
        super(MyEmbedding, self).__init__(**kwargs)
        self.dims = dims
        self.out_put_dim = out_put_dim
        
        with self.name_scope():
            for i in range(len(dims)):
                temp_embedding = gnn.Embedding(input_dim=dims[i], output_dim=out_put_dim)
                self._children.append(temp_embedding)
                
    def forward(self, x):
        """
        :type x: 
        :param x: 
        :return: 
        """
        length = len(self.dims)
        embeding_list = []
        for i in range(length):
            temp = x[:, i]
            embedding_temp = self._children[i](temp)
            embeding_list.append(embedding_temp)
        
        result = embeding_list[0]
        for i in range(1, len(self.dims)):
            result = mx.nd.concat(result, embeding_list[i], dim=1)
        return result#.reshape((x.shape[0], x.shape[1], self.out_put_dim)) 

In [156]:
net =  MyEmbedding(input_dims, 10)
net.initialize()
net.dims

[3, 3, 4, 3]

In [157]:
y = net(train_data)

y


[[-0.02504851 -0.01044503  0.01866136 -0.03754863 -0.02486168 -0.04009686
   0.01664888  0.00150746  0.03394407  0.009599    0.04330547 -0.0547312
  -0.0028558  -0.04849725  0.04821662 -0.06353974 -0.06074558 -0.04670204
  -0.02848735  0.01206998  0.04700147 -0.06282886  0.06697129 -0.04774661
   0.0100894  -0.04393635 -0.01526375  0.01134419 -0.06307098 -0.04359851
  -0.00819998  0.03835732 -0.03207801  0.04146395  0.02158369  0.01324444
  -0.06485769  0.03891881  0.05490585  0.0674106 ]
 [ 0.02128588  0.03395962 -0.0178537   0.05038493  0.01998015 -0.03694049
  -0.02614833 -0.03305176 -0.02443967  0.0642269   0.03303897  0.02016263
  -0.06500886  0.01683573 -0.04787604  0.04198996  0.06533863  0.01854012
  -0.06098125  0.05454382  0.01867548  0.05417627 -0.03297959 -0.06340497
  -0.02115871  0.05473544  0.04678871  0.03667305 -0.02143921  0.01324476
  -0.02765403 -0.03756863  0.01625396  0.02061453 -0.03916054  0.0029757
   0.03975413  0.00606338  0.02886308 -0.02679005]
 [-0.025048

In [158]:
net.collect_params()

myembedding3_ (
  Parameter myembedding3_embedding0_weight (shape=(3, 10), dtype=<class 'numpy.float32'>)
  Parameter myembedding3_embedding1_weight (shape=(3, 10), dtype=<class 'numpy.float32'>)
  Parameter myembedding3_embedding2_weight (shape=(4, 10), dtype=<class 'numpy.float32'>)
  Parameter myembedding3_embedding3_weight (shape=(3, 10), dtype=<class 'numpy.float32'>)
)

In [185]:
whole_net = gnn.Sequential()
with whole_net.name_scope():
    whole_net.add(MyEmbedding(input_dims, 5))
    whole_net.add(gnn.Dense(1, activation='sigmoid'))


In [186]:
whole_net.initialize()


In [187]:
whole_net.collect_params()

sequential15_ (
  Parameter sequential15_myembedding0_embedding0_weight (shape=(3, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential15_myembedding0_embedding1_weight (shape=(3, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential15_myembedding0_embedding2_weight (shape=(4, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential15_myembedding0_embedding3_weight (shape=(3, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential15_dense0_weight (shape=(1, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential15_dense0_bias (shape=(1,), dtype=<class 'numpy.float32'>)
)

In [206]:
#whole_net[4].weights
whole_net[1].weight

Parameter sequential15_dense0_weight (shape=(1, 20), dtype=<class 'numpy.float32'>)

In [188]:
train_data


[[ 0.  0.  0.  0.]
 [ 1.  1.  1.  1.]
 [ 0.  0.  2.  0.]
 [ 1.  0.  0.  0.]]
<NDArray 4x4 @cpu(0)>

In [189]:
whole_net(train_data)


[[ 0.50218391]
 [ 0.50110292]
 [ 0.50202662]
 [ 0.50149661]]
<NDArray 4x1 @cpu(0)>

In [190]:
train_data


[[ 0.  0.  0.  0.]
 [ 1.  1.  1.  1.]
 [ 0.  0.  2.  0.]
 [ 1.  0.  0.  0.]]
<NDArray 4x4 @cpu(0)>

In [195]:
loss = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)

In [196]:
trainer = gluon.Trainer(whole_net.collect_params(), 'sgd', {'learning_rate': 0.3})

In [197]:
from mxnet import autograd
for _ in range(10):
    with autograd.record():
        output = whole_net(train_data)
        loss_num = loss(label_data, output) 
        print(mx.ndarray.sum(loss_num))
    loss_num.backward()
    trainer.step(4)


[ 47.03222656]
<NDArray 1 @cpu(0)>

[ 34.58136749]
<NDArray 1 @cpu(0)>

[ 28.32457161]
<NDArray 1 @cpu(0)>

[ 24.39650154]
<NDArray 1 @cpu(0)>

[ 17.16625595]
<NDArray 1 @cpu(0)>

[ 2.51093793]
<NDArray 1 @cpu(0)>

[ 0.83573377]
<NDArray 1 @cpu(0)>

[ 0.5463323]
<NDArray 1 @cpu(0)>

[ 0.40614247]
<NDArray 1 @cpu(0)>

[ 0.32239342]
<NDArray 1 @cpu(0)>


In [198]:
whole_net(train_data)


[[ 0.99871945]
 [ 0.00482073]
 [ 0.99820781]
 [ 0.99824548]]
<NDArray 4x1 @cpu(0)>

In [184]:
loss(label_data, output) 


[-1.02472532  0.69314718 -1.00947654 -1.01635683]
<NDArray 4 @cpu(0)>

In [54]:
label_data


[ 1.  0.  1.  1.]
<NDArray 4 @cpu(0)>

In [55]:
output


[[ 0.75407493]
 [ 0.75407493]
 [ 0.75407493]
 [ 0.75407493]]
<NDArray 4x1 @cpu(0)>

In [32]:
import math

1 / (1 + math.exp(0.78))

0.31431988606174616

In [35]:
- math.log2(0.31431988606174616)

1.6696945449425675

In [59]:
whole_net._children[0]._children[0]

sequential4_myembedding0_embedding0_ (
  Parameter sequential4_myembedding0_embedding0_weight (shape=(3, 10), dtype=<class 'numpy.float32'>)
)

In [172]:
def predict(x):
    return 1 / (1 + mx.nd.exp(-whole_net(x)))

In [171]:
predict(mx.nd.array([1, 2, 0]))


[ 0.7310586   0.88079703  0.5       ]
<NDArray 3 @cpu(0)>

In [176]:
predict(train_data)


[[ 0.99162138]
 [ 0.95811266]
 [ 0.99046141]
 [ 0.99070919]]
<NDArray 4x1 @cpu(0)>

In [ ]:
a = mx.nd.array([[1, 2], [3, 4], [5, 6]])

In [72]:
i = 1
a[:, i].reshape((3, 1))


[[ 2.]
 [ 4.]
 [ 6.]]
<NDArray 3x1 @cpu(0)>

In [38]:
a[:, 1]


[ 2.  4.  6.]
<NDArray 3 @cpu(0)>

In [49]:
c = mx.nd.concat(a[:, 0].reshape((3, 1)), a[:, 1].reshape((3, 1)), dim=1)
c


[[ 1.  2.]
 [ 3.  4.]
 [ 5.  6.]]
<NDArray 3x2 @cpu(0)>

In [48]:
c.reshape((3, 2, 1))


[[[ 1.]
  [ 2.]]

 [[ 3.]
  [ 4.]]

 [[ 5.]
  [ 6.]]]
<NDArray 3x2x1 @cpu(0)>